<div style="display:fill;
           background-color:#F2E5E5;
           letter-spacing:0.5px;border-bottom: 2px solid black;">
<img src="https://raw.githubusercontent.com/IqmanS/Machine-Learning-Notebooks/refs/heads/main/Land_Cover_EUROSAT/thumb.jpg">
<H1 style="padding: 20px; color:black; font-weight:600; font-family: 'Garamond', 'Lucida Sans', sans-serif; text-align: center; font-size: 38px;">Land-Cover Classification with EuroSAT Dataset </H1>
</div>


In [ ]:
import os
import shutil
import random
from tqdm import tqdm

import numpy as np
import pandas as pd
import re

import PIL
import seaborn as sns
import matplotlib.pyplot as plt
from os import listdir
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
# import imutils
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Flatten, Activation, Dense, MaxPooling2D, Dropout#, RandomRotation, RandomFlip, RandomContrast
from tensorflow.keras import Sequential

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
# from keras.models import Model
# from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
# from keras.optimizers import Adam


from keras.applications import VGG16, VGG19, MobileNetV2 
from keras.applications import ResNet50, ResNet50V2, ResNet152V2, ResNet101V2
from keras.applications import InceptionV3, Xception

from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, fbeta_score, accuracy_score, f1_score, precision_score, recall_score

# Data Exploration

In [ ]:
IMAGE_DIR = "../input/2750"
LABELS = os.listdir(IMAGE_DIR)
print(LABELS)

In [ ]:
fig, axs = plt.subplots(6, 6,figsize=(12,12),dpi=500)
for i in range(6):
    for j in range(6):
        label = np.random.choice(LABELS)
        number = np.random.randint(1000,2000)
        img_path = IMAGE_DIR+"/"+label+"/"+label+"_"+str(number)+".jpg"
        img = PIL.Image.open(img_path, 'r')
        axs[i, j].imshow(img)
        axs[i, j].axis("off")

for ax in axs.ravel():
    ax.set_axis_off();
    
plt.subplots_adjust(wspace=0, hspace=0);
# plt.tight_layout();
fig.patch.set_facecolor('black')
plt.savefig("image.png");
plt.show();

In [ ]:
def augment_data(file_dir, save_to_dir, n_generated_samples = 2):
    data_gen = ImageDataGenerator(rotation_range=10, 
                                  width_shift_range=0.1, 
                                  height_shift_range=0.1, 
                                  shear_range=0.1, 
                                  brightness_range=(0.5, 1.0),
                                  horizontal_flip=True, 
                                  vertical_flip=True, 
                                  fill_mode='nearest'
                                 )

    for filename in listdir(file_dir):
        image = cv2.imread(file_dir + '/' + filename)
        # reshape the image
        image = image.reshape((1,)+image.shape)
        save_prefix = 'aug_' + filename[:-4]
        i=0
        for batch in data_gen.flow(x=image, batch_size=1, save_to_dir=save_to_dir,save_prefix=save_prefix, save_format='jpg'):
                i += 1
                if i > n_generated_samples:
                    break

In [ ]:
# plot class distributions of whole dataset
counts = {}

for l in LABELS:
    counts[l] = len(os.listdir(os.path.join(IMAGE_DIR, l)))

    
plt.figure(figsize=(12, 6))

plt.bar(range(len(counts)), list(counts.values()), align='center')
plt.xticks(range(len(counts)), list(counts.keys()), fontsize=12, rotation=40)
plt.xlabel('class label', fontsize=13)
plt.ylabel('class size', fontsize=13)
plt.title('EUROSAT Class Distribution', fontsize=15);

# Data Preprocessing

In [ ]:
# SET SEED
np.random.seed(6)
seed = np.random.randint(123)
print(seed)

In [ ]:
TRAIN_DIR = '../working/training'
TEST_DIR = '../working/testing'
BATCH_SIZE = 128
NUM_CLASSES = len(LABELS)
INPUT_SHAPE = (64, 64, 3)
CLASS_MODE = 'categorical'
TEST_SIZE = 0.25

for path in (TRAIN_DIR, TEST_DIR):
    if not os.path.exists(path):
        os.mkdir(path)

for l in LABELS:
    if not os.path.exists(os.path.join(TRAIN_DIR, l)):
        os.mkdir(os.path.join(TRAIN_DIR, l))

    if not os.path.exists(os.path.join(TEST_DIR, l)):
        os.mkdir(os.path.join(TEST_DIR, l))

In [ ]:
# map each image path to their class label in 'data'
data = {}

for l in LABELS:
    for img in os.listdir(IMAGE_DIR+'/'+l):
        data.update({os.path.join(IMAGE_DIR, l, img): l})

X = pd.Series(list(data.keys()))
y = pd.get_dummies(pd.Series(data.values()))

split = StratifiedShuffleSplit(n_splits=1, test_size=TEST_SIZE, random_state=seed)

# split the list of image paths
for train_idx, test_idx in split.split(X, y):
    
    train_paths = X[train_idx]
    test_paths = X[test_idx]

    # define a new path for each image depending on training or testing
    new_train_paths = [re.sub('\.\.\/input\/2750', '../working/training', i) for i in train_paths]
    new_test_paths = [re.sub('\.\.\/input\/2750', '../working/testing', i) for i in test_paths]

    train_path_map = list((zip(train_paths, new_train_paths)))
    test_path_map = list((zip(test_paths, new_test_paths)))
    
    # move the files
    # print("moving training files..")
    for i in tqdm(train_path_map,desc = "Moving Training Data"):
        if not os.path.exists(i[1]):
            if not os.path.exists(re.sub('training', 'testing', i[1])):
                shutil.copy(i[0], i[1])
    
    # print("moving testing files..")
    for i in tqdm(test_path_map,desc = "Moving Testing Data"):
        if not os.path.exists(i[1]):
            if not os.path.exists(re.sub('training', 'testing', i[1])):
                shutil.copy(i[0], i[1])

In [ ]:
# Create a ImageDataGenerator Instance which can be used for data augmentation

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=60,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=(0.5, 1.0),
    horizontal_flip=True, 
    vertical_flip=True, 
    fill_mode='nearest'
)

train_generator = train_gen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=(64, 64),
    batch_size=BATCH_SIZE,
    class_mode=CLASS_MODE,
    color_mode='rgb',
    shuffle=True,
    seed=seed
)

# test generator for evaluation purposes with no augmentations, just rescaling
test_gen = ImageDataGenerator(
    rescale=1./255,
)

test_generator = test_gen.flow_from_directory(
    directory=TEST_DIR,
    target_size=(64, 64),
    batch_size=BATCH_SIZE,
    class_mode=CLASS_MODE,
    color_mode='rgb',
    shuffle=False,
    seed=seed
)

pred_generator = test_gen.flow_from_directory(  #no batches
    directory=TEST_DIR,
    target_size=(64, 64),
    batch_size=1,
    class_mode=None,
    color_mode='rgb',
    shuffle=False,
    seed=seed
)

In [ ]:
print(train_generator.class_indices)

In [ ]:
np.save('class_indices', train_generator.class_indices)

# Building Deep Learning Models

1. Custom CNN
2. MobileNetV2
3. ResNet101V2


In [ ]:
N_STEPS = train_generator.samples//BATCH_SIZE
N_VAL_STEPS = test_generator.samples//BATCH_SIZE
N_EPOCHS = 100

In [ ]:
def plot_history(history):
       
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    plt.figure(figsize=(20, 5))
    plt.subplot(1, 2, 1)
    plt.plot(acc)
    plt.plot(val_acc)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.title("Acc vs Val Acc")
    
    plt.subplot(1, 2, 2)
    plt.plot(loss)
    plt.plot(val_loss)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.title("Loss vs Val Loss")
    
    plt.show();

In [ ]:
early_stop = EarlyStopping(monitor='val_accuracy',
                           patience=4,
                           restore_best_weights=True,
                           mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,
                              patience=3, min_lr=0.00001)

## 1. Custom CNN Model

In [ ]:
def build_model(input_shape):
    model = Sequential()
    model.add(Input(input_shape))
    model.add(Conv2D(16,5,activation="relu"))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(32,5,activation="relu"))
    model.add(Conv2D(32,5,activation="relu"))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(64,7,activation="relu"))
    model.add(MaxPooling2D(2,2))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dropout(0.2))
    model.add(Dense(len(LABELS),activation="softmax"))
    return model

custom_model = build_model(INPUT_SHAPE)

In [ ]:
train_generator.reset()
test_generator.reset()

custom_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
custom_model_history = custom_model.fit_generator(train_generator, steps_per_epoch=N_STEPS, epochs=50,
                                                  callbacks=[early_stop, reduce_lr],
                                                  validation_data=test_generator, validation_steps=N_VAL_STEPS)

In [ ]:
plot_history(custom_model_history)

# 2. Transfer Learning MobileNetV2 

In [ ]:
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam

In [ ]:
def build_mobilenet(input_shape, fine_tune=None):
    conv_base = MobileNetV2(include_top=False,
                             weights='imagenet', 
                             input_shape=input_shape)
    base_model = conv_base.output
    base_model = Flatten()(base_model)
    base_model = Dense(1000)(base_model)
    base_model = Dropout(0.2)(base_model)
    output_layer = Dense(len(LABELS),activation="softmax")(base_model) #o/p layer
    model = Model(inputs=conv_base.input, outputs=output_layer)
    if type(fine_tune) == int:
        for layer in conv_base.layers[fine_tune:]: layer.trainable = True
    else:
        for layer in conv_base.layers: layer.trainable = False
    return model

mobilenet = build_mobilenet(INPUT_SHAPE,20)

train_generator.reset()
test_generator.reset()

mobilenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
mobilenet_history = mobilenet.fit_generator(train_generator, steps_per_epoch=N_STEPS, epochs=50,
                                            callbacks=[early_stop, reduce_lr],
                                            validation_data=test_generator, validation_steps=N_VAL_STEPS)

In [ ]:
plot_history(mobilenet_history)

# 3. Transfer Learning ResNet101V2 

In [ ]:
def build_resnet101(input_shape ,fine_tune=None):
    conv_base = ResNet101V2(include_top=False,
                             weights='imagenet', 
                             input_shape=input_shape)
    base_model = conv_base.output
    base_model = Flatten()(base_model)
    base_model = Dense(1000)(base_model)
    base_model = Dropout(0.2)(base_model)
    output_layer = Dense(len(LABELS),activation="softmax")(base_model) #o/p layer
    model = Model(inputs=conv_base.input, outputs=output_layer)

    if type(fine_tune) == int:
        for layer in conv_base.layers[fine_tune:]: layer.trainable = True
    else:
        for layer in conv_base.layers: layer.trainable = False
    
    return model

resnet = build_resnet101(INPUT_SHAPE,20)

train_generator.reset()
test_generator.reset()

resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_history = resnet.fit_generator(train_generator, steps_per_epoch=N_STEPS, epochs=50,
                                      callbacks=[early_stop, reduce_lr],
                                      validation_data=test_generator, validation_steps=N_VAL_STEPS)

In [ ]:
plot_history(resnet_history)

# Evaluation

In [ ]:
def display_results(y_true, y_preds, class_labels):
    
    results = pd.DataFrame(precision_recall_fscore_support(y_true, y_preds),
                          columns=class_labels).T
    results.rename(columns={0: 'Precision',
                           1: 'Recall',
                           2: 'F-Score',
                           3: 'Support'}, inplace=True)
    
    conf_mat = pd.DataFrame(confusion_matrix(y_true, y_preds), 
                            columns=class_labels,
                            index=class_labels)    
    accuracy = accuracy_score(y_true, y_preds)
    f1 = f1_score(y_true, y_preds,average='macro')
    precision = precision_score(y_true, y_preds,average='macro')
    recall = recall_score(y_true, y_preds,average='macro')
    
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1}")    
    print(f"Precision Score: {precision}")
    print(f"Recall Score: {recall}")  
    return results, conf_mat

def plot_predictions(y_true, y_preds, test_generator, class_indices):

    fig = plt.figure(figsize=(20, 12))
    for i, idx in enumerate(np.random.choice(test_generator.samples, size=20, replace=False)):
        ax = fig.add_subplot(4, 5, i + 1, xticks=[], yticks=[])
        ax.imshow(np.squeeze(test_generator[idx]))
        pred_idx = np.argmax(y_preds[idx])
        true_idx = y_true[idx]
                
        plt.tight_layout()
        ax.set_title("{}\n({})".format(class_indices[pred_idx], class_indices[true_idx]),
                     color=("green" if pred_idx == true_idx else "red"))    

## Evaluating Custom CNN Model

In [ ]:
class_indices = train_generator.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())
pred_generator.reset()

predictions = custom_model.predict_generator(pred_generator, steps=len(pred_generator.filenames))
predicted_classes = np.argmax(np.rint(predictions), axis=1)
true_classes = pred_generator.classes

prf, conf_mat = display_results(true_classes, predicted_classes, class_indices.values())
prf

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(conf_mat,annot=True,fmt='.0f');
plt.title("Conf Matrix for Custom CNN Model");

In [ ]:
plot_predictions(true_classes, predictions, pred_generator, class_indices)

## Evaluating MobileNetV2 Model

In [ ]:
class_indices = train_generator.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())
pred_generator.reset()

predictions = mobilenet.predict_generator(pred_generator, steps=len(pred_generator.filenames))
predicted_classes = np.argmax(np.rint(predictions), axis=1)
true_classes = pred_generator.classes

prf, conf_mat = display_results(true_classes, predicted_classes, class_indices.values())
prf

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(conf_mat,annot=True,fmt='.0f');
plt.title("Conf Matrix for MobileNetV2");

In [ ]:
plot_predictions(true_classes, predictions, pred_generator, class_indices)
plt.savefig("sample_output.png")

## Evaluating ResNet101V2 Model

In [ ]:
class_indices = train_generator.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())
pred_generator.reset()

predictions = resnet.predict_generator(pred_generator, steps=len(pred_generator.filenames))
predicted_classes = np.argmax(np.rint(predictions), axis=1)
true_classes = pred_generator.classes

prf, conf_mat = display_results(true_classes, predicted_classes, class_indices.values())
prf

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(conf_mat,annot=True,fmt='.0f');
plt.title("Conf Matrix for ResNet101V2 Model");

In [ ]:
plot_predictions(true_classes, predictions, pred_generator, class_indices)
plt.savefig("sample_output.png")